In [1]:
using Distributed

In [2]:
addprocs(4)

4-element Vector{Int64}:
 2
 3
 4
 5

In [3]:
x = workers()

4-element Vector{Int64}:
 2
 3
 4
 5

In [11]:
@everywhere using DistributedArrays

In [12]:
D = DArray([@spawnat p [(woker = p, flag = Ref(false))] for p in workers()])

4-element DArray{@NamedTuple{woker::Int64, flag::Base.RefValue{Bool}}, 1, Vector{@NamedTuple{woker::Int64, flag::Base.RefValue{Bool}}}}:
 (woker = 2, flag = Base.RefValue{Bool}(false))
 (woker = 3, flag = Base.RefValue{Bool}(false))
 (woker = 4, flag = Base.RefValue{Bool}(false))
 (woker = 5, flag = Base.RefValue{Bool}(false))

In [13]:
[@fetchfrom p localindices(D) for p in workers()]

4-element Vector{Tuple{UnitRange{Int64}}}:
 (1:1,)
 (2:2,)
 (3:3,)
 (4:4,)

In [14]:
[@fetchfrom p localpart(D) for p in workers()]

4-element Vector{Vector{@NamedTuple{woker::Int64, flag::Base.RefValue{Bool}}}}:
 [(woker = 2, flag = Base.RefValue{Bool}(false))]
 [(woker = 3, flag = Base.RefValue{Bool}(false))]
 [(woker = 4, flag = Base.RefValue{Bool}(false))]
 [(woker = 5, flag = Base.RefValue{Bool}(false))]

In [15]:
for p in workers()
    @fetchfrom p only(localpart(D)).flag[]=true
end

In [16]:
D

4-element DArray{@NamedTuple{woker::Int64, flag::Base.RefValue{Bool}}, 1, Vector{@NamedTuple{woker::Int64, flag::Base.RefValue{Bool}}}}:
 (woker = 2, flag = Base.RefValue{Bool}(true))
 (woker = 3, flag = Base.RefValue{Bool}(true))
 (woker = 4, flag = Base.RefValue{Bool}(true))
 (woker = 5, flag = Base.RefValue{Bool}(true))

In [17]:
D[2].flag[]

true

In [27]:
t = @async begin
    sleep(10)
    1
end

Task (runnable) @0x00000001120b1f90

In [28]:
t

Task (runnable) @0x00000001120b1f90

In [29]:
istaskdone(t)

false

In [30]:
while true
    println("Waiting")
    if timedwait(()->istaskdone(t), 100) == :ok
        println("Done waiting: $(fetch(t))")
        break
    end
end

Waiting
Done waiting: 1


In [31]:
fetch(t)

1

In [32]:
using Distributed

In [50]:
@everywhere function test1()
    @async while true
        w=workers()
        println("workers=$(w)")
        break
    end
end
@everywhere test2() = println("myid=$(myid())")
@everywhere function test3()
    println("myid=$(myid()), running test2")
    remote_do(test2, 2)
end

@everywhere function test4(channel)
    println("taking from channel")
    x = take!(channel)
    println("x=$(x)")
end

In [35]:
test1()

workers=[2, 3, 4, 5, 6, 7, 8, 9]


In [36]:
test2()

myid=1


In [37]:
test3()

myid=1, running test2
      From worker 2:	myid=2


In [55]:
remote_do(test1, 2)

      From worker 2:	workers=[2, 3, 4, 5, 6, 8, 9, 10, 11]


In [39]:
remote_do(test3, 3)

      From worker 3:	myid=3, running test2
      From worker 2:	myid=2


In [40]:
addprocs(2)

2-element Vector{Int64}:
 10
 11

In [11]:
remote_do(test1, 3)

      From worker 3:	workers=[3, 2, 4, 5, 6, 7]


In [41]:
ch = RemoteChannel(()->Channel{Int64}(32))

RemoteChannel{Channel{Int64}}(1, 1, 156)

In [42]:
put!(ch, 1)

RemoteChannel{Channel{Int64}}(1, 1, 156)

In [43]:
workers()

10-element Vector{Int64}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11

In [44]:
remote_do(test4, 6, ch)

      From worker 6:	taking from channel
      From worker 6:	x=1


In [45]:
rmprocs(7)

Task (done) @0x0000000113630be0

In [46]:
workers()

9-element Vector{Int64}:
  2
  3
  4
  5
  6
  8
  9
 10
 11

In [48]:
stop_thread = false
@async while true
    if stop_thread
        break
    end
    x = workers()
    println("workers=$(x)")
    sleep(2)
end

workers=[2, 3, 4, 5, 6, 8, 9, 10, 11]


Task (runnable) @0x0000000149f8ecb0

workers=[2, 3, 4, 5, 6, 8, 9, 10, 11]
workers=[2, 3, 4, 5, 6, 8, 9, 10, 11]
workers=[2, 3, 4, 5, 6, 8, 9, 10, 11]
workers=[2, 3, 4, 5, 6, 8, 9, 10, 11]
workers=[2, 3, 4, 5, 6, 8, 9, 10, 11]


In [49]:
stop_thread=true

true